# Resnet
Flexible code using Resnet architecture that can be trained on different datasets

In [1]:
import os, sys
import numpy as np
import importlib
import utils; importlib.reload(utils)
from utils import *

from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import scipy.misc
np.set_printoptions(suppress=True)

%matplotlib inline

Using cuDNN version 7003 on context None
Mapped name None to device cuda0: GeForce GTX 1080 Ti (0000:01:00.0)
/opt/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#reference to imp directories
current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir+'/data/dogscats/'

In [3]:
path=DATA_HOME_DIR

In [4]:
%cd $DATA_HOME_DIR

#Set path to sample/ path if desired
models_path = DATA_HOME_DIR + 'models/resnet/'
test_path = DATA_HOME_DIR + '/test/' #Using all the test data
results_path=DATA_HOME_DIR + '/results/'
train_path=path + '/train/'
valid_path=path + '/valid/'

/home/hearth/ML/course/deeplearning1/nbs/data/dogscats


In [5]:
batch_size = 64

In [6]:
def assign_batches(shuffle, batch_size, target_size):
    return (get_batches(train_path, batch_size=batch_size, shuffle=shuffle, target_size=target_size),
get_batches(valid_path, batch_size=batch_size, shuffle=shuffle, target_size=target_size),
get_batches(test_path, batch_size=batch_size, shuffle=shuffle, target_size=target_size))

In [7]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 22998 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


## Resnet
Resnet is unique in that it is made of Resnet blocks where, in each block there is conv layers in succession but in the end of a resnet block, there is an addition with an identity matrix. Maxpooling is done after this to conform with matrix dimension requirements. 



In [8]:
%cd $LESSON_HOME_DIR
import resnet50; importlib.reload(resnet50)
from resnet50 import Resnet50
%cd $DATA_HOME_DIR

/home/hearth/ML/course/deeplearning1/nbs
/home/hearth/ML/course/deeplearning1/nbs/data/dogscats


In [9]:
rn0 = Resnet50(include_top=False, size=(400,400)).model
rn0.output_shape[1:]

ValueError: Negative dimension size caused by subtracting 3 from 2 for 'max_pooling2d_1/MaxPool' (op: 'MaxPool') with input shapes: [?,2,200,64].

In [ ]:
batches, val_batches, test_batches = assign_batches(shuffle=False, 
                                        batch_size=batch_size, target_size=(400,400))

### Resnet on smaller images

In [ ]:
#Setting include top to false means only include convolutional layers and
#the additional fully connected blocks will be ignored. 
rn0 = Resnet50(include_top=False).model

In [ ]:
rn0.output_shape[1:]

In [ ]:
batches, val_batches, test_batches = assign_batches(shuffle=False, 
                                        batch_size=batch_size, target_size=(224,224))

#### If RAM>32

In [ ]:
val_features = rn0.predict_generator(val_batches, np.ceil(val_batches.samples/batch_size))

In [ ]:
trn_features = rn0.predict_generator(batches, np.ceil(batches.samples/batch_size))

In [ ]:
trn_labels[0]

In [ ]:
trn_features= load_array(models_path + 'trn_rn0_conv.bc')
val_features = load_array(models_path +'val_rn0_conv.bc')

### Precompute features

[Optional] Clean existing saved convolution features if required

In [ ]:
%cd $models_path
%rm -R *.dat
%cd $DATA_HOME_DIR

#### If RAM<32

In [ ]:
fname = models_path+'trn_rn0_conv.dat'
for i in range(batches.n // batch_size+1):
    conv_feat = rn0.predict_on_batch(batches.next()[0])
    if not i:
        c = bcolz.carray(conv_feat, rootdir=fname, mode='a')
    else:
        c.append(conv_feat)
    c.shape
c.flush()

In [ ]:
fname = models_path+'val_rn0_conv.dat'
for i in range(val_batches.n // batch_size+1):
    conv_val_feat = rn0.predict_on_batch(val_batches.next()[0])
    if not i:
        c = bcolz.carray(conv_val_feat, rootdir=fname, mode='a')
    else:
        c.append(conv_val_feat)
c.flush()

In [ ]:
fname = models_path+'val_rn0_conv_y.dat'
save_array(fname, val_labels)
fname = models_path+'trn_rn0_conv_y.dat'
save_array(fname, trn_labels)

In [ ]:
X=bcolz.open(models_path+'trn_rn0_conv.dat', mode='r')
y= bcolz.open(models_path+'trn_rn0_conv_y.dat', mode='r')

In [ ]:
valX=bcolz.open(models_path+'val_rn0_conv.dat', mode='r')
valy= bcolz.open(models_path+'val_rn0_conv_y.dat', mode='r')

In [ ]:
trn_batches=BcolzArrayIterator(X,y, batch_size=batch_size, shuffle=True)
val_batches=BcolzArrayIterator(valX,valy, batch_size=batch_size, shuffle=True)

## Fully connected layer

In [ ]:
trn_batches.N

In [ ]:
def get_fc_layers(p):
    return [
        BatchNormalization(axis=1, input_shape=rn0.output_shape[1:]),
        Flatten(),
        Dropout(p),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(2, activation='softmax')
    ]

In [ ]:
fc_model = Sequential(get_fc_layers(0.8))
fc_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
fc_model.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
fc_model.fit_generator(trn_batches, epochs=5, steps_per_epoch=np.ceil(trn_batches.N/batch_size), validation_data=val_batches, 
                     validation_steps=int(np.ceil(val_batches.N/batch_size)))

In [ ]:
fc_model.fit(trn_features, trn_labels, epochs=5, batch_size=batch_size, validation_data=(val_features, val_labels))

## Global Average Pooling Layer

It takes in as output the 2048 13x13 (dogs-cats) output of convolution layers and does average pooling on all of them essentially putting a single number on all features which represents a feature corresponding to a label. eg: a feature map learning to see cat tails will respond with a higher number to a cat and make for a good input to final Dense layer. 

Advantages:
1. No dropouts because very minimal dense layers - Also generalise better because lot less 
parameters.
2. Average pooling might work better in a usecases like cats and dogs where subject spans the whole image. 
3. Resnet used average pooling in its original version and it makes sense to leverage the benefits provided by the weights trained on a dataset with average pooling switched on.

In [ ]:
 def get_ap_layers(p):
        return [
            GlobalAveragePooling2D(input_shape=rn0.output_shape[1:]),
            Dropout(p),
            Dense(2, activation='softmax')
        ]

In [ ]:
model = Sequential(get_ap_layers(0.01))

In [ ]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(trn_batches, epochs=5, steps_per_epoch=np.ceil(trn_batches.N/batch_size), validation_data=val_batches, 
                     validation_steps=int(np.ceil(val_batches.N/batch_size)))

## Making predictions on individual samples

In [ ]:
def predict_on_single(img):
    print(img.shape)
    plt.imshow(img, interpolation='nearest')
    plt.show()
    img = scipy.misc.imresize(img, (400, 400))
    img = img.reshape([1, 3, 400, 400], order='C')
    conv_feat = rn0.predict(img)
    prob = model.predict(conv_feat, batch_size=None, verbose=0)
    return (prob)

In [ ]:
img = mpimg.imread(path+'/dog.1.jpg')
predict_on_single(img)